In [1]:
from pyspark import SparkConf
from pyspark import SparkContext

In [2]:
conf = SparkConf()
conf.setMaster('spark://ip-172-31-9-200:7077')
conf.setAppName('spark_analytics_chpt_3')
conf.set("spark.executor.memory", "10g")
sc = SparkContext(conf=conf)

Data from http://www-etud.iro.umontreal.ca/~bergstrj/audioscrobbler_data.html

In [3]:
user_artist_data = sc.textFile('user_artist_data.txt', 12)
artist_data = sc.textFile('artist_data.txt')
artist_alias = sc.textFile('artist_alias.txt')

In [4]:
user_artist_data.take(10)

[u'1000002 1 55',
 u'1000002 1000006 33',
 u'1000002 1000007 8',
 u'1000002 1000009 144',
 u'1000002 1000010 314',
 u'1000002 1000013 8',
 u'1000002 1000014 42',
 u'1000002 1000017 69',
 u'1000002 1000024 329',
 u'1000002 1000025 1']

In [5]:
artist_data.take(10)

[u'1134999\t06Crazy Life',
 u'6821360\tPang Nakarin',
 u'10113088\tTerfel, Bartoli- Mozart: Don',
 u'10151459\tThe Flaming Sidebur',
 u'6826647\tBodenstandig 3000',
 u'10186265\tJota Quest e Ivete Sangalo',
 u'6828986\tToto_XX (1977',
 u'10236364\tU.S Bombs -',
 u'1135000\tartist formaly know as Mat',
 u'10299728\tKassierer - Musik f\xfcr beide Ohren']

In [6]:
artist_alias.take(10)

[u'1092764\t1000311',
 u'1095122\t1000557',
 u'6708070\t1007267',
 u'10088054\t1042317',
 u'1195917\t1042317',
 u'1112006\t1000557',
 u'1187350\t1294511',
 u'1116694\t1327092',
 u'6793225\t1042317',
 u'1079959\t1000557']

In [7]:
user_artist_data.map(lambda x: int(x.split(' ')[0])).stats()

(count: 24296858, mean: 1947573.26535, stdev: 496000.544975, max: 2443548.0, min: 90.0)

In [8]:
user_artist_data.map(lambda x: int(x.split(' ')[1])).stats()

(count: 24296858, mean: 1718704.09376, stdev: 2539389.04017, max: 10794401.0, min: 1.0)

In [9]:
artist_data.map(lambda x: x.split('\t')).take(10)

[[u'1134999', u'06Crazy Life'],
 [u'6821360', u'Pang Nakarin'],
 [u'10113088', u'Terfel, Bartoli- Mozart: Don'],
 [u'10151459', u'The Flaming Sidebur'],
 [u'6826647', u'Bodenstandig 3000'],
 [u'10186265', u'Jota Quest e Ivete Sangalo'],
 [u'6828986', u'Toto_XX (1977'],
 [u'10236364', u'U.S Bombs -'],
 [u'1135000', u'artist formaly know as Mat'],
 [u'10299728', u'Kassierer - Musik f\xfcr beide Ohren']]

In [10]:
artist_alias.map(lambda x: x.split('\t')).take(10)

[[u'1092764', u'1000311'],
 [u'1095122', u'1000557'],
 [u'6708070', u'1007267'],
 [u'10088054', u'1042317'],
 [u'1195917', u'1042317'],
 [u'1112006', u'1000557'],
 [u'1187350', u'1294511'],
 [u'1116694', u'1327092'],
 [u'6793225', u'1042317'],
 [u'1079959', u'1000557']]

In [11]:
from pyspark.mllib import recommendation

In [12]:
recommendation.Rating?